In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [2]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [15]:
df = load_meta()
df = df.loc[~df['sample'].str.contains('_')]
df['lab_sample'] = df['lab_number_sample'].astype(str)+'_'+\
                      df['lab_sampleid'].astype(str)+'_'+\
                      df['cell_line_id'].astype(str)
df = df[['cell_line_id', 'sample', 'hapmap_DNA_ID', 'lab_sample', 'population']].drop_duplicates()

temp_df = pd.read_csv('../snakemake/transcript_discovery_personal/cell_line_ids.txt', header=None, names=['cell_line_id'])

# make a 1000g cell line id col
df['cell_line_id_1000g'] = df.cell_line_id

inds = df.loc[~df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())].index
df.loc[inds, 'cell_line_id_1000g'] = df.loc[inds, 'hapmap_DNA_ID']
len(df.index)

# limit to just those in 1000g
df = df.loc[df.cell_line_id_1000g.isin(temp_df.cell_line_id.tolist())]
assert len(df.index) == 30

# TODO bad sample that hasn't finished on espresso
bad_samples = ['NA19328']
df = df.loc[~df.cell_line_id_1000g.isin(bad_samples)]

hap = ['hap1', 'hap2']

meta_df = df.copy(deep=True)

In [16]:
df.head()

,cell_line_id,sample,hapmap_DNA_ID,lab_sample,population,cell_line_id_1000g
0,HG04217,ITU5,no_hapmap,39_IN5_HG04217,ITU,HG04217
1,HG04216,ITU4,no_hapmap,38_IN4_HG04216,ITU,HG04216
2,HG03732,ITU3,no_hapmap,37_IN3_HG03732,ITU,HG03732
3,HG03729,ITU2,no_hapmap,36_IN2_HG03729,ITU,HG03729
4,HG03719,ITU1,no_hapmap,35_IN1_HG03719,ITU,HG03719


## How does IC sharing differ for the different genotypes?

In [23]:
file = proc_cfg(config['lr']['td_personal']['cerb']['ic_summary'],od)
df = pd.read_csv(file, sep='\t')

# remove chromosomes that aren't represented in normal hg38 alignmnts
hg38_chrs = df.loc[df.genome=='hg38', 'Chromosome'].unique().tolist()
print(len(df.index))
df = df.loc[df.Chromosome.isin(hg38_chrs)]
print(len(df.index))

4549058
4549056


In [26]:
df['ic_id'] = df.Chromosome.astype(str)+'_'+\
              df.Strand.astype(str)+'_'+\
              df.Coordinates.astype(str)
df.head()

,Unnamed: 0,Chromosome,Strand,Coordinates,Name,genome,cell_line_id,ic_id
0,0,GL000195.1,-,86517-49092-48954-44923,ENSG00000276256_1,hg38,HG04217,GL000195.1_-_86517-49092-48954-44923
1,1,GL000195.1,-,86517-49119-48954-44923,ENSG00000276256_2,hg38,HG04217,GL000195.1_-_86517-49119-48954-44923
2,2,GL000195.1,-,86517-74180-74120-49119-48954-44923,ENSG00000303853_1,hg38,HG04217,GL000195.1_-_86517-74180-74120-49119-48954-44923
3,3,GL000195.1,-,86517-83958,ENSG00000303853_2,hg38,HG04217,GL000195.1_-_86517-83958
4,4,GL000195.1,-,86517-84116,ENSG00000303853_3,hg38,HG04217,GL000195.1_-_86517-84116


In [50]:
# what percentage of ics by cell line (personal haplotypes) 
# are found also by the reference genome?
big_df = pd.DataFrame()
for cl in df.cell_line_id.unique():
    temp = df.loc[df.cell_line_id==cl]
    temp = temp[['ic_id', 'genome']]
    temp['hg38'] = temp['hap1'] = temp['hap2'] = False
    temp.loc[temp.genome=='hg38', 'hg38'] = True
    temp.loc[temp.genome=='hap1', 'hap1'] = True
    temp.loc[temp.genome=='hap2', 'hap2'] = True

    temp = temp[['ic_id', 'hg38', 'hap1', 'hap2']].groupby('ic_id').max().reset_index()
    # temp['personal'] = temp.hap1|temp.hap2
    temp = temp.groupby(['hg38', 'hap1', 'hap2']).nunique().reset_index()
    temp.rename({'ic_id':'n_ic'}, axis=1, inplace=True)
    temp['total'] = temp['n_ic'].sum()
    temp['perc'] = (temp.n_ic/temp.total)*100
    
    # one haplotype at a time
    temp2 = temp[['hg38', 'hap1', 'n_ic', 'total']]
    temp2 = temp2.groupby(['hg38', 'hap1', 'total']).sum().reset_index()
    temp2['personal_genotype'] = 'hap1'
    temp2.rename({'hap1': 'personal_genotype'}, axis=1, inplace=True)
    temp2

    # one haplotype at a time
    temp3 = temp[['hg38', 'hap2', 'n_ic', 'total']]
    temp3 = temp3.groupby(['hg38', 'hap2', 'total']).sum().reset_index()
    temp3['personal_genotype'] = 'hap2'
    temp3.rename({'hap2': 'personal_genotype'}, axis=1, inplace=True)
    temp3

    temp2 = pd.concat([temp2, temp3], axis=0)
    temp2['perc'] = (temp2['n_ic']/temp2['total'])*100
    temp2
    
    temp2['cell_line_id'] = cl
    big_df = pd.concat([temp2, big_df], axis=0)

In [51]:
temp = temp[['ic_id', 'genome']]
temp['hg38'] = temp['hap1'] = temp['hap2'] = False
temp.loc[temp.genome=='hg38', 'hg38'] = True
temp.loc[temp.genome=='hap1', 'hap1'] = True
temp.loc[temp.genome=='hap2', 'hap2'] = True

temp = temp[['ic_id', 'hg38', 'hap1', 'hap2']].groupby('ic_id').max().reset_index()
# temp['personal'] = temp.hap1|temp.hap2
temp = temp.groupby(['hg38', 'hap1', 'hap2']).nunique().reset_index()
temp.rename({'ic_id':'n_ic'}, axis=1, inplace=True)
temp['total'] = temp['n_ic'].sum()
temp['perc'] = (temp.n_ic/temp.total)*100
temp

,hg38,hap1,hap2,n_ic,total,perc
0,False,False,True,176,55711,0.315916
1,False,True,False,160,55711,0.287196
2,False,True,True,2007,55711,3.602520
3,True,False,False,88,55711,0.157958
4,True,False,True,122,55711,0.218987
5,True,True,False,108,55711,0.193858
6,True,True,True,53050,55711,95.223564


In [55]:
# one haplotype at a time
temp2 = temp[['hg38', 'hap1', 'n_ic', 'total']]
temp2 = temp2.groupby(['hg38', 'hap1', 'total']).sum().reset_index()
temp2['personal_genotype'] = 'hap1'
temp2.rename({'hap1': 'personal_genotype'}, axis=1, inplace=True)
temp2

# one haplotype at a time
temp3 = temp[['hg38', 'hap2', 'n_ic', 'total']]
temp3 = temp3.groupby(['hg38', 'hap2', 'total']).sum().reset_index()
temp3['personal_genotype'] = 'hap2'
temp3.rename({'hap2': 'personal_genotype'}, axis=1, inplace=True)
temp3

temp2 = pd.concat([temp2, temp3], axis=0)
temp2['perc'] = (temp2['n_ic']/temp2['total'])*100
temp2

,hg38,personal_genotype,total,n_ic,personal_genotype,perc
0,False,False,55711,176,hap1,0.315916
1,False,True,55711,2167,hap1,3.889717
2,True,False,55711,210,hap1,0.376945
3,True,True,55711,53158,hap1,95.417422
0,False,False,55711,160,hap2,0.287196
1,False,True,55711,2183,hap2,3.918436
2,True,False,55711,196,hap2,0.351816
3,True,True,55711,53172,hap2,95.442552


## Do we get more NNC (novel splice site) when we look at the personal haploytpes versus the hg38?